In [59]:
using Missings
using StatsKit
using MLJ
import BetaML
import AlgebraOfGraphics as AOG
using CairoMakie

set_theme!(theme_dark())
ENV["DATAFRAMES_ROWS"] = 8

8

##### Data Cleaning

In [60]:
function fix_missings(passengerData::DataFrame)::DataFrame
    mode_replace(column) = coalesce.(column, mode(column)) |> categorical
    dropmissing!(passengerData, :PassengerId)
    transform!(passengerData, :HomePlanet => mode_replace => :HomePlanet)
    transform!(passengerData, :CryoSleep => mode_replace => :CryoSleep)
    transform!(passengerData, :Destination => mode_replace => :Destination)
    transform!(passengerData, :VIP => mode_replace => :VIP)

    mean_replace(column) = coalesce.(column, mean(skipmissing(column)))
    transform!(passengerData, :Age => mean_replace => :Age)

    select!(passengerData, Not(:Name))
    return passengerData
end

function combine_totals(passengerData::DataFrame)::DataFrame
    zeroSpend(column) = coalesce.(column, 0)
    transform!(passengerData, :RoomService => zeroSpend => :RoomService)
    transform!(passengerData, :FoodCourt => zeroSpend => :FoodCourt)
    transform!(passengerData, :ShoppingMall => zeroSpend => :ShoppingMall)
    transform!(passengerData, :Spa => zeroSpend => :Spa)
    transform!(passengerData, :VRDeck => zeroSpend => :VRDeck)
    passengerData.TotalSpent = passengerData.RoomService +
                               passengerData.FoodCourt + passengerData.ShoppingMall +
                               passengerData.Spa + passengerData.VRDeck
    select!(passengerData, Not([:RoomService, :FoodCourt,
        :ShoppingMall, :Spa, :VRDeck]))
    return passengerData
end


function split_columns(passengerData::DataFrame)::DataFrame
    transform!(passengerData, :Cabin => ByRow(cabin -> coalesce(cabin, "//")) => :Cabin)
    passengerData.RoomDeck = [
        String(deck[1]) for deck in split.(passengerData.Cabin, "/")
    ] |> categorical
    passengerData.RoomSide = [
        String(deck[3]) for deck in split.(passengerData.Cabin, "/")
    ] |> categorical
    select!(passengerData, Not(:Cabin))
    return passengerData
end

passengerData = CSV.read("data/train.csv", DataFrame) |>
                fix_missings |>
                split_columns |>
                combine_totals

Row,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,RoomDeck,RoomSide,TotalSpent
,String7,Cat…,Cat…,Cat…,Float64,Cat…,Bool,Cat…,Cat…,Float64
1,0001_01,Europa,false,TRAPPIST-1e,39.0,false,false,B,P,0.0
2,0002_01,Earth,false,TRAPPIST-1e,24.0,false,true,F,S,736.0
3,0003_01,Europa,false,TRAPPIST-1e,58.0,true,false,A,S,10383.0
4,0003_02,Europa,false,TRAPPIST-1e,33.0,false,false,A,S,5176.0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
8690,9278_01,Earth,true,PSO J318.5-22,18.0,false,false,G,S,0.0
8691,9279_01,Earth,false,TRAPPIST-1e,26.0,false,true,G,S,1873.0
8692,9280_01,Europa,false,55 Cancri e,32.0,false,false,E,S,4637.0
8693,9280_02,Europa,false,TRAPPIST-1e,44.0,false,true,E,S,4826.0


##### Exploration Analysis

In [61]:
describe(passengerData)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,PassengerId,,0001_01,,9280_02,0,String7
2,HomePlanet,,Earth,,Mars,0,"CategoricalValue{String7, UInt32}"
3,CryoSleep,,false,,true,0,"CategoricalValue{Bool, UInt32}"
4,Destination,,55 Cancri e,,TRAPPIST-1e,0,"CategoricalValue{String15, UInt32}"
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
7,Transported,0.503624,false,1.0,true,0,Bool
8,RoomDeck,,,,T,0,"CategoricalValue{String, UInt32}"
9,RoomSide,,,,S,0,"CategoricalValue{String, UInt32}"
10,TotalSpent,1440.87,0.0,716.0,35987.0,0,Float64


In [62]:
transported_deck = AOG.data(passengerData) *
                   AOG.mapping(:HomePlanet, color=:Transported, dodge=:Transported) *
                   AOG.frequency()
AOG.draw(transported_deck)

MethodError: MethodError: findall(::AlgebraOfGraphics.var"#8#9", ::Vector{Any}) is ambiguous.

Candidates:
  findall(el::T, cont::Array{T}; returnTuple) where T
    @ BetaML.Utils ~/.julia/packages/BetaML/V0tAN/src/Utils/Processing.jl:71
  findall(testf::F, A::AbstractArray) where F<:Function
    @ Base array.jl:2359
  findall(testf::Function, A)
    @ Base array.jl:2355

Possible fix, define
  findall(::F, ::Array{T}) where {F<:Function, T<:Function}


##### Statistical Approach

In [63]:
fn = @formula(Transported ~ VIP + CryoSleep + Age + TotalSpent)
logitModel = glm(fn, passengerData, Bernoulli(), LogitLink())

function measureAccuracy(model, passengerData::DataFrame)::Float64
    predictions = GLM.predict(model, passengerData)
    predictions = [
        if x < 0.5
            false
        else
            true
        end for x in predictions
    ]
    prediction_df = DataFrame(y_actual=passengerData.Transported, y_predicted=predictions, prob_predicted=predictions)
    prediction_df.correctly_classified = prediction_df.y_actual .== prediction_df.y_predicted
    accuracy = mean(prediction_df.correctly_classified)

    return accuracy

end

acc = measureAccuracy(logitModel, passengerData)

0.718279075117911

##### Machine Learning Approach

In [64]:
passengerData = MLJ.coerce!(passengerData,
    MLJ.autotype(passengerData)
)

y, X = MLJ.unpack(passengerData,
    ==(:Transported), !=([:PassengerID, :RoomService, :FoodCourt,
        :ShoppingMall, :Spa, :VRDeck]),
    colname -> true)
train, test = MLJ.partition(eachindex(y), 0.7; shuffle=true)
Tree = BetaML.DecisionTreeClassifier()
mach = machine(Tree, X, y; scitype_check_level=0)
MLJ.fit!(mach, rows=train)
pred = MLJ.predict(mach, X[test, :])
yhat = StatsBase.mode.(pred)
MLJ.accuracy(yhat, y[test])


┌ Info: Training machine(DecisionTreeClassifier(max_depth = 0, …), …).
└ @ MLJBase /Users/zachary.drake/.julia/packages/MLJBase/5cxU0/src/machines.jl:492


0.7074386503067485

##### Predictions

In [71]:
predictData = CSV.read("data/test.csv", DataFrame) |>
              fix_missings |>
              split_columns |>
              combine_totals

pred = MLJ.predict(mach, predictData)
predictData.Transported = StatsBase.mode.(pred)

select!(predictData, :PassengerId, :Transported)

function bool_to_str(column)
    for i in column
        println(i)
        if column == true
            return "True"
        elseif column == false
            return "False"
        end
    end
end

transform!(predictData, :Transported => bool_to_str => :Transported)

# CSV.write("output/submission.csv", predictData)

CategoricalValue{Bool, UInt32}[true, false, true, false, false, false, true, true, true, false, false, false, false, false, false, false, false, true, false, false, true, false, true, false, false, false, false, true, false, true, true, false, true, false, false, false, true, false, false, true, false, false, false, true, false, true, false, false, false, false, false, true, false, true, true, false, false, false, true, false, true, false, false, true, true, false, true, false, true, true, false, false, true, false, true, false, true, true, true, true, true, true, false, false, false, false, true, true, true, true, false, false, false, false, false, false, true, true, false, false, true, false, true, false, true, false, false, true, false, true, true, false, false, false, false, true, true, true, false, false, false, false, false, false, false, true, false, false, false, false, true, false, false, false, true, true, false, false, false, false, true, true, true, false, true, false, true

, true, false, true, true, false, false, false, true, false, false, true, false, false, false, false, false, false, false, true, true, false, false, false, false, false, true, true, false, false, false, false, true, false, true, true, false, false, false, true, true, false, true, false, false, false, false, true, false, false, true, false, false, true, false, false, true, false, false, false, false, false, true, false, true, true, true, true, true, false, false, true, false, false, true, true, false, false, false, true, false, false, true, true, false, true, true, false, false, false, true, false, false, true, true, true, true, false, true, true, false, true, true, true, false, true, false, false, false, true, false, true, false, true, true, true, true, false, false, false, false, true, false, true, false, false, false, false, false, true, true, true, false, false, true, true, false, true, false, true, true, false, true, true, true, false, true, true, true, false, false, false, false, 

 false, false, true, true, false, true, true, false, true, true, false, false, true, true, true, false, false, false, true, true, false, false, true, true, false, true, false, false, false, false, true, true, true, false, false, false, false, false, false, true, true, true, true, false, true, true, true, true, true, true, true, true, false, true, true, true, true, false, true, false, false, false, false, false, true, false, true, false, false, false, true, true, true, true, true, true, true, true, false, true, true, false, false, false, false, true, false, true, false, true, true, false, false, true, false, false, true, false, true, false, true, true, false, false, false, false, true, true, false, true, true, false, false, true, false, true, false, true, true, false, false, false, false, false, false, false, true, false, true, false, false, true, true, false, false, false, false, false, true, true, false, false, true, false, false, false, false, true, true, true, true, true, true, fals

 false, true, true, true, true, true, true, false, true, true, true, true, false, false, false, true, true, false, false, false, false, true, true, false, true, true, true, false, true, false, true, true, false, false, false, false, false, true, false, false, true, false, false, true, true, true, true, true, false, false, true, true, true, true, true, true, true, false, true, true, true, false, true, true, true, false, true, false, false, false, false, false, true, true, false, true, false, false, false, true, false, false, false, false, false, true, true, true, true, true, true, false, false, true, false, true, false, true, true, true, true, true, false, false, true, false, false, false, false, false, false, true, true, true, true, false, true, true, true, true, true, false, false, true, false, true, true, true, true, false, false, true, true, false, false, false, false, true, false, false, true, true, false, false, true, false, true, false, true, false, false, false, true, true, true

, true, true, false, true, true, true, false, false, false, false, false, false, false, true, false, false, true, false, false, false, true, true, true, true, true, true, false, false, true, false, false, true, true, false, false, true, true, false, true, false, false, true, false, true, false, true, false, false, false, true, false, true, true, true, true, false, true, false, true, true, false, false, false, false, false, false, false, true, true, true, true, false, true, false, true, true, true, false, false, false, true, false, true, false, true, true, false, false, false, true, false, false, false, false, true, false, true, true, true, true, false, true, true, true, false, true, false, true, false, false, false, true, true, false, true, false, false, true, true, true, false, false, true, false, true, true, true, false, false, true, true, true, false, false, false, false, false, true, false, false, true, false, true, false, true, true, false, false, false, false, false, true, true, 

...

 true, true, true, true, false, true, false, false, false, false, true, false, false, true, false, false, false, false, false, false, true, true, true, true, true, true, false, false, true, true, true, true, false, false, true, true, true, false, false, true, false, true, true, false, false, false, true, false, false, true, false, false, false, false, false, false, false, true, true, false, false, false, false, false, true, true, false, false, false, false, true, false, true, true, false, false, false, true, true, false, true, false, false, false, false, true, false, false, true, false, false, true, false, false, true, false, false, false, false, false, true, false, true, true, true, true, true, false, false, true, false, false, true, true, false, false, false, true, false

, false, true, true, false, true, true, false, false, false, true, false, false, true, true, true, true, false, true, true, false, true, true, true, false, true, false, false, false, true, false, true, false, true, true, true, true, false, false, false, false, true, false, true, false, false, false, false, false, true, true, true, false, false, true, true, false, true, false, true, true, false, true, true, true, false, true, true, true, false, false, false, false, true, true, false, true, true, false, false, true, true, true, true, true, true, true, false, false, false, false, true, true, true, false, false, true, false, true, true, true, false, true, true, false, false, true, true, true, true, true, false, false, false, true, false, true, true, false, true, false, true, true, true, true, false, true, false, true, true, true, false, false, false, true, true, false, false, false, true, false, false, true, false, true, false, false, true, true, false, true, true, true, true, false, false

 true, true, false, true, false, true, true, false, false, true, false, true, false, false, false, true, true, false, true, true, false, false, true, false, false, true, true, false, true, false, true, true, true, true, true, false, false, false, true, true, false, true, true, false, false, false, true, false, true, true, true, false, false, false, true, true, false, false, true, false, false, true, true, false, true, true, true, true, false, false, true, true, false, false, true, true, false, true, true, false, true, true, true, false, true, true, true, false, true, true, false, false, true, true, false, false, false, true, false, false, true, true, true, true, true, false, true, true, false, true, false, true, false, false, true, false, true, true, false, true, false, true, true, false, false, false, false, false, true, true, false, true, false, false, true, true, false, true, true, false, true, false, true, false, false, true, false, true, false, true, false, true, true, false, fals

Excessive output truncated after 524290 bytes.

 false, true, false, false, false, false, false, true, true, true, true, true, true, false, false, true, false, true, false, true, true, true, true, true, false, false, true, false, false, false, false, false, false, true, true, true, true, false, true, true, true, true, true, false, false, true, false, true, true, true, true, false, false, true, true, false, false, false, false, true, false, false, true, true, false, false, true, false, true, false, true, false, false, false, true, true, true, false, true, false, false, false, true, true, false, true, false, true, true, false, false, true, true, false, true, true, true, false, true, false, true, false, false, false, false, true, true, false, false, false, true, true, false, true, false, true, true, true, false, true, true, true, false, true, false, true, true, true, false, true, true, true, false, true, false, true, true, true, false, false, false, false, true, true, false, false, false, true, false, false, false, true, true, false, t